In [1]:
import glob
import pandas as pd

from os.path import basename
import pickle

from level2.level2_utils import *
from level2.level2_roi_extraction import level2_roi_extractor
from level2.level2_roi_extraction import load_rois, get_roi_data_for_beta
from level2.level2_roi_extraction import get_roi_data_for_l2_betas, get_roi_data_for_multirun_l2_betas
#import modules from files in a parallel directory "direct_regression"


from direct_regression.get_all_series import get_beta_img, get_roi_data, get_moment_trial_type_revealed, get_behavioral_data_with_moment_trial_type_revealed, mask_3d_subject_image
from direct_regression.get_all_series import get_all_subj_df
from direct_regression.fmri_utils import *



In [2]:

config = load_config("direct_regression/config.yml")
analysis_name = 'conditions'


#get the masks
mask_locations = [
    config['mask_location'] + 'aim3/masks/neurosynth/',
    config['mask_location'] + 'aim3/masks/'
]

signature_locations = [
    config['mask_location'] + 'signatures/aim3/'
]

roi_df = get_mask_df_from_mask_locations(mask_locations)
signature_df = get_mask_df_from_mask_locations(signature_locations)
roi_df['image_type'] = 'mask'
signature_df['image_type'] = 'signature'

#combine the two dfs
roi_df = pd.concat([roi_df, signature_df])
#get the list of raw nii files
glob_path = config['fmriprep_dir'] + config['nii_raw_path']

Benjamins-MacBook-Pro-2.local
{'dev_scripts_path': '/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/', 'level2_output_path': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/SST/', 'rural_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'rubric_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/DEV_scoring_rubrics', 'l2_analysis_files': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/level2/', 'input_mat_path': '/Users/benjaminsmith/oregon/code/DEV_scripts/fMRI/fx/multiconds/SST/full_duration/conditions', 'dropbox_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/analysis_files/data/', 'dev_analysis_data_dir': '/Users/benjaminsmith/Dropbox (University of Oregon)/UO-SAN Lab/Berkman Lab/Devaluation/ana

## SST

In [3]:
# #filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
# sst_roi_df = roi_df.loc[roi_df['mask_label'].str.contains('striatum|finger|motor|response inhibition')]

# ## SST
# train_betas_with_data = get_data_for_confirmed_train_subjs(
#     beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/wave1/conditions/sub-DEV*/",
#     nonbids_data_path = config['nonbids_data_path'],
#     #ml_data_folderpath = ml_data_folderpath,
#     ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
#     dropbox_datapath=config['dropbox_data_dir'],
#     exclude_test_subjs=False
# )
# train_betas_with_data['wave']=1

# #we're not interestd in getting contrasts; comment this out.
# #betas_with_contrasts = get_contrasts_for_betas(train_betas_with_data)
# betas_with_paths = get_beta_fnames_for_beta_dirs(train_betas_with_data[0:5])


### W1 W2 analysis

In [4]:
#get the masks
mask_locations = [
    config['mask_location'] + 'aim3/masks/neurosynth/',
    config['mask_location'] + 'aim3/masks/',
    config['mask_location'] + 'striatum/',
    config['mask_location']
]
wave_compare_roi_df = get_mask_df_from_mask_locations(mask_locations)

In [5]:
#filter the mask_label in mask_df, using regex, to only use stiraum, finger movements, motor control, and response inbhitioin
sst_roi_df = wave_compare_roi_df.loc[wave_compare_roi_df['mask_label'].str.contains('response inhibition|striatum_joint_reward|value')]


In [6]:

## SST
train_betas_with_data = get_data_for_confirmed_task_session(
    beta_glob = config['nonbids_data_path'] + "fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV*/",
    nonbids_data_path = config['nonbids_data_path'],
    #ml_data_folderpath = ml_data_folderpath,
    ml_scripting_path = config['dev_scripts_path'] + "/fMRI/ml",
    dropbox_datapath=config['dropbox_data_dir'],
    subj_wave_inclusion='all'
)

train_betas_with_data['wave']='all'

loaded 275 rows from data_by_ppt.csv
479 sessions with no reported problems in task Redcap scanner notes, added to the provision useable_dev_id list.
7 subjects excludeable for missing scan data.
470 sessions remaining on the provision useable_dev_id list from the redcap list after excluding subjects with missing scan data.
470 sessions remaining on the provisional useable_dev_id list from the redcap list after excluding subjects excluded by motion quality process.
beta paths before exclusion: 250


/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:392: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useable_betas_with_data.sort_values('subject_id', inplace=True)
/var/folders/cj/4mb6t1f906j397tj71pxfxz00000gn/T/ipykernel_11291/374063233.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_betas_with_data['wave']='all'


In [9]:
len(train_betas_with_data)

159

In [10]:
contrasts_with_paths = get_contrasts_for_betas(train_betas_with_data)

/Users/benjaminsmith/Google Drive/oregon/code/DEV_scripts/fMRI/fx/models/SST/level2/level2_utils.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  betas['spm_l2_targeted_contrast_filepath'] = None


/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_conditions/sub-DEV014/
0: Healthy_Go spmT_0001.nii
1: Healthy_NoGo spmT_0002.nii
2: Unhealthy_Go spmT_0003.nii
3: Unhealthy_NoGo spmT_0004.nii
4: Null_Cue spmT_0005.nii
5: Stop(Healthy>Unhealthy) spmT_0006.nii
6: Stop(Unhealthy>Healthy) spmT_0007.nii
7: Go(Healthy>Unhealthy) spmT_0008.nii
8: Go(Unhealthy>Healthy) spmT_0009.nii
9: Healthy_Go(W2-W1) spmT_0010.nii
10: Healthy_Go(W1-W2) spmT_0011.nii
11: Healthy_NoGo(W2-W1) spmT_0012.nii
12: Healthy_NoGo(W1-W2) spmT_0013.nii
13: Unhealthy_Go(W2-W1) spmT_0014.nii
14: Unhealthy_Go(W1-W2) spmT_0015.nii
15: Unhealthy_NoGo(W2-W1) spmT_0016.nii
16: Unhealthy_NoGo(W1-W2) spmT_0017.nii
17: Null_Cue(W2-W1) spmT_0018.nii
18: Null_Cue(W1-W2) spmT_0019.nii
19: Stop(Healthy>Unhealthy)(W2-W1) spmT_0020.nii
20: Stop(Healthy>Unhealthy)(W1-W2) spmT_0021.nii
/Users/benjaminsmith/Google Drive/oregon/data/DEV/nonbids_data/fMRI/fx/models/SST/all_waves/health_cond

### Spatial normalization

In [11]:
#from level2.level2_roi_extraction import level2_roi_extractor

contrast_name_list = [
    'Unhealthy_NoGo(W2-W1)',
    'Unhealthy_Go(W2-W1)',
    'Stop(Healthy>Unhealthy)(W2-W1)'
    ]

l2_roi_extractor = level2_roi_extractor(center_data=False,scale_data=False)
l2_roi_extractor.load_all_images_simultaneously=True
l2_roi_extractor.image_standardize = True

#get the ROI data
roi_data_sst_conditions = l2_roi_extractor.get_roi_data_for_l2_contrasts(contrasts_with_paths, contrast_name_list, sst_roi_df)

response inhibition_association-test_z_FDR_0.01
producing the vector for this mask/series...
value_association-test_z_FDR_0.01
producing the vector for this mask/series...
striatum_joint_reward_mask
producing the vector for this mask/series...
value_association_frontal_medial_cortex
producing the vector for this mask/series...
Unhealthy_NoGo(W2-W1)
loading 159images...
...loaded.
['DEV014', 'DEV015', 'DEV017', 'DEV019', 'DEV020', 'DEV021', 'DEV023', 'DEV028', 'DEV029', 'DEV030', 'DEV031', 'DEV033', 'DEV038', 'DEV040', 'DEV041', 'DEV042', 'DEV043', 'DEV044', 'DEV046', 'DEV047', 'DEV051', 'DEV052', 'DEV053', 'DEV054', 'DEV055', 'DEV056', 'DEV057', 'DEV058', 'DEV059', 'DEV060', 'DEV062', 'DEV063', 'DEV065', 'DEV066', 'DEV067', 'DEV068', 'DEV070', 'DEV071', 'DEV072', 'DEV073', 'DEV074', 'DEV075', 'DEV076', 'DEV077', 'DEV078', 'DEV079', 'DEV081', 'DEV083', 'DEV084', 'DEV085', 'DEV086', 'DEV087', 'DEV089', 'DEV090', 'DEV093', 'DEV094', 'DEV096', 'DEV097', 'DEV100', 'DEV103', 'DEV104', 'DEV10

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


proportion of image active: 0.01383226975097848
activity scalar is [ 0.22419931 -0.32014032  0.1351386   0.22543307 -0.25205148  0.19454515
  0.19306284 -0.30743318  0.10900483  0.11580914  0.16817942 -0.25951954
  0.11483765  0.15055591  0.115246   -0.00983386  0.0736598   0.13726426
  0.22957503 -0.07427096  0.29349982  0.41390836 -0.06454978  0.10668345
  0.16724936 -0.15868848  0.03666732  0.1561561   0.20275465 -0.01190542
 -0.17018358 -0.10711326  0.18425352 -0.28662363  0.36605753  0.24731954
  0.00893852 -0.03069774  0.36886061 -0.05368259 -0.25427982  0.19929002
 -0.33672739  0.04961802  0.02834214 -0.11746637 -0.09340802 -0.28776721
 -0.05893263 -0.14919323  0.03992526  0.29623585  0.33728979  0.34728573
  0.04393933  0.12576112 -0.1008639  -0.03534128 -0.01018278 -0.05632971
  0.04545994 -0.21276695 -0.48211112 -0.18435758 -0.1978834  -0.07009918
 -0.1861345   0.01275243 -0.08490313 -0.32079367  0.24302427 -0.0856375
  0.09198122 -0.26242748 -0.19868003 -0.05425131 -0.377803

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(niimg, result, niimg.affine)


proportion of image active: 0.016650108360635286
activity scalar is [-0.00754983 -0.30309204 -0.39423665 -0.11746919 -0.21250778  0.40447196
 -0.2974102   0.10398036 -0.27486762  0.2110842   0.13364695 -0.05313795
  0.03594365  0.33856686 -0.01174474 -0.0863494  -0.07976029 -0.25580497
  0.26534358  0.0997541   0.52942079 -0.03281623 -0.04064452 -0.23311491
  0.38430649 -0.21457395  0.01705319  0.29495181  0.02334501  0.10721725
 -0.54447242 -0.11385465  0.43020328  0.13359609  0.14650879 -0.168846
 -0.16630066 -0.30363545  0.20077269  0.09997339  0.10073793  0.20203076
 -0.20008569  0.20556781  0.17959914  0.45185677  0.04596346 -0.06809553
  0.12291505 -0.15431408  0.40546273  0.33351492  0.29103013 -0.39660242
  0.11898664 -0.14228463  0.28645463  0.210292    0.20488969  0.00130636
 -0.04173237  0.05127119 -0.06138474  0.38791019 -0.11928393 -0.00862886
 -0.23424746  0.66461291 -0.09899057 -0.65271573 -0.17035337  0.09197299
  0.05708246 -0.1168783   0.07243008 -0.34940333 -0.344814

...

proportion of image active: 0.016650108360635286
activity scalar is [-0.44804054 -0.02345837 -0.02950399  0.31003974 -0.15918314 -0.45808749
  0.03316491  0.04710697  0.37900889 -0.06418052  0.30032029  0.04034263
  0.04749467 -0.41107829  0.13420091 -0.0049266  -0.28467743  0.13911777
 -0.05804594 -0.03462235  0.20119964  0.35351088  0.40925098  0.0255907
 -0.39376769  0.02593441 -0.31174387  0.18164084 -0.09867498  0.12223965
  0.1994686  -0.44304093  0.07875935  0.02511799  0.0624821  -0.00825613
 -0.4509983  -0.08697934  0.16601022  0.31895271  0.05285368  0.01333166
 -0.15513071 -0.11453687 -0.03569606 -0.10079931  0.19997024 -0.06241095
 -0.30866824 -0.08593883  0.05351754 -0.11073012 -0.03377296 -0.38592224
 -0.03761932 -0.30195281  0.0516178  -0.04078579 -0.25413046  0.02101564
 -0.06804344  0.06874605 -0.05106115 -0.1007317   0.34738304 -0.48170448
  0.18929361  0.1168682   0.09822092 -0.10252583 -0.17898271 -0.12940802
 -0.30985389 -0.08549474  0.15131841 -0.12499128  0.01902

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:545: UserWarning: Casting data from int32 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:274: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  ret

proportion of image active: 0.005345483279191523
activity scalar is [-0.12177748 -0.12070618 -0.07894907  0.05402987 -0.12283523 -0.2269132
  0.27467693 -0.01899716  0.21506547 -0.72757182  0.28662025  0.02509438
 -0.02815512 -0.19130274 -0.14919618  0.2891224  -0.01138629 -0.11697701
 -0.21767045 -0.12860157  0.28056037  0.43208311  0.13858036 -0.02823381
 -0.34444235 -0.10085008 -0.15480495 -0.02678751  0.08382325  0.07628078
 -0.377724   -0.25629771 -0.01927529 -0.29505853  0.55135273 -0.16937654
  0.0896274  -0.06393346 -0.11352565 -0.30922425  0.28036557 -0.15518913
  0.08475774 -0.18160472  0.06113579  0.24435192 -0.29663808 -0.00614003
  0.02548067 -0.22830323 -0.39019872 -0.05805371  0.15154338 -0.5306005
 -0.10218243 -0.0029567   0.07331781 -0.32470486 -0.06936019  0.06366373
  0.03699188 -0.39480667 -0.00488903 -0.20542518  0.198211   -0.04769087
  0.16859263  0.33810919 -0.20009009 -0.16849746 -0.1112152  -0.32504981
 -0.12223719 -0.3192963   0.03046312 -0.27658287  0.121773

/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:545: UserWarning: Casting data from int32 to float32
  warnings.warn("Casting data from %s to %s" % (data.dtype.name, aux))
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/resampling.py:274: UserWarning: Resampling binary images with continuous or linear interpolation. This might lead to unexpected results. You might consider using nearest interpolation instead.
  warnings.warn("Resampling binary images with continuous or "
/Users/benjaminsmith/opt/anaconda3/envs/neuralsignature/lib/python3.10/site-packages/nilearn/image/image.py:1041: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  ret

proportion of image active: 0.0007864810288022106
activity scalar is [-1.61032048  0.62118094  0.06280271  0.78563706 -0.2843863  -0.98811708
  0.34715269 -0.07064975  0.44753204  0.29544148  0.66433466  0.29353577
  0.45936927 -0.46394892  0.11929149 -0.91279497  0.05411993  0.1509323
  0.12533674 -0.5095096   0.29533486  0.7804029   0.61041894  0.09016543
 -0.76147438 -0.54893781 -0.78047291  0.60871619 -0.37750403  0.46797975
  0.99629326 -0.80259083  0.12082635  0.24772672  0.15261792 -0.02947592
 -1.70565196 -0.44227298  0.62079844  1.03001692 -0.64628924  0.82513524
 -0.3398604   0.59031501  0.02670373 -1.03576647  1.37551804 -0.11052871
 -1.36039027 -0.54831275 -0.08281981 -0.79569431  0.08474337 -0.27899229
 -0.28981685 -0.96181407 -0.14906066  0.3420733  -0.21885532  0.31063553
 -0.28029916  0.5284787   0.02373297  0.35915806  0.53531553 -0.96166139
 -0.26349801  0.47783118  0.30816949 -0.00245025  0.55749551  0.0769427
 -0.83725629 -0.08785128  0.6289291  -0.45448935 -0.35014

In [12]:
roi_data_sst_conditions.to_csv(config['dropbox_data_dir'] + '/subject_sst_health_avg_roi_data_raw_zscored.csv')

See this doc to demonstrate why we need to normalize across the image, not normalize across the subjects:

https://docs.google.com/spreadsheets/d/15iykB7qzCm8xqt7hgzIH58e-pAWgGz9BWKjEeN0MpBw/edit#gid=0


# TEMP